<a href="https://colab.research.google.com/github/iGhostlp/Albus/blob/Gunter-y-Ernesto/Proyecto_BBVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Armado del entorno

In [3]:
# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [4]:
# Unzip the file
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [5]:
!readlink -f $(which java) | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [6]:
# Set up the environment for Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [7]:
# Install library for finding Spark
!pip install -q findspark

# Import the libary
import findspark

# Initiate findspark
findspark.init()

In [8]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder.master("local[*]").config('spark.sql.parquet.datetimeRebaseModeInRead','CORRECTED').getOrCreate()

# Check Spark Session Information
spark

# Extraccion de datos desde parquet, clientes y teléfonos.

In [9]:
df_customer = spark.read.load('sample_data/customer_basics_bootcamp.snappy.parquet', sep=',', inferschema='true', header='true')
df_phones = spark.read.load('sample_data/phones_bootcamp.snappy.parquet', sep=',', inferschema='true', header='true')


In [10]:
df_customer_phones = df_customer.join(df_phones, 'customer_id', how='inner')

In [11]:
df_customer_phones.show()

+-----------+-------------+-------------+--------------+---------------+--------------------+---------+----------------+-----------+-----------------------+-----------------------+----------+-------------------+-----------------+--------------------------+-----------+----------------+-----------------+-------------------+-----------+----------+----------------------+--------------------+---------------------+-----------+--------------------------+----------------------------+-------------------------------+------------------------+------------------------+--------------------------+------------------------+--------------------------+----------------------------+------------------+------------------------+-------------------+-------------------+---------------------------+-----------------+-------------------+-----------------+-------------------------------+--------------------+---------------------+------------------+----------------+--------------------+--------------------------+---

QUIERO: Filtrar el DataFrame de contactos telefónicos de clientes y resguardar los 3 contactos más actuales por cliente.   
PARA: Reducir el volumen de datos y trabajar solo con los más actualizados

registry_entry_date Momento en el que se realiza el alta de un registro
last_change_date Fecha en el que se registra en el sitema un cambio en la informacion

In [12]:
df_phones.sort('customer_id').show()

+-----------+--------------+-------------------+-----------------+----------+----------------+---------------+-------------+-------------------+-----------------+-------------+-------------+---------------+-----------------------+------------------+---------------------+-----------------+----------+-----------+--------------------------+---------------------+--------------------+---------------------+----------------+-------------------------+-------------------------+------------------+-------------------+-----------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+---------------------+
|customer_id|phone_use_type|address_sequence_id|phone_sequence_id|phone_type|phone_country_id|prefix_phone_id|phone_area_id|cellphone_

In [13]:
from pyspark.sql.functions import row_number, desc

In [14]:
from pyspark.sql.window import Window

#Filtro telefonos



In [15]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat,col, row_number, desc, collect_list
from pyspark.sql.window import Window

Cortamos la tabla, las columnas que no consideramos parte del analisis

In [16]:
df_phones_cut = df_phones.drop('phone_intern_id','phone_country_id', 'aditional_info_txt_desc', 'primary_phone_type','address_sequence_type','address_town_name','zipcode_id', 'province_id','sender_application_id','normalization_status_type','normalization_reason_name','validity_start_date','validity_end_date','dlvy_day_monday_type','dlvy_day_tuesday_type','dlvy_day_wednesday_type','dlvy_day_thursday_type','dlvy_day_friday_type','dlvy_day_friday_type','dlvy_day_saturday_type','delivery_contact_start_hm_date','delivery_contact_end_hm_date','operational_load_date','normalization_date')

In [17]:
df_phones_sorted = df_phones_cut.orderBy([df_phones_cut.customer_id, desc('last_change_date')])
df_phones_sorted.show()

+-----------+--------------+-------------------+-----------------+----------+---------------+-------------+-------------------+-----------------+-------------+-------------+--------------------------+---------------------+--------------------+----------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+
|customer_id|phone_use_type|address_sequence_id|phone_sequence_id|phone_type|prefix_phone_id|phone_area_id|cellphone_prefix_id|phone_exchange_id|phone_line_id|     phone_id|customer_phone_status_type|phone_status_mod_date|contact_channel_type|wrong_phone_type|registry_entry_date|register_user_id|last_change_date|last_change_user_id|last_change_hms_date|last_change_terminal_id|
+-----------+--------------+-------------------+-----------------+----------+---------------+-------------+-------------------+-----------------+-------------+-------------+--------------------------+---------------------+------------------


Concatenamos las columnas del numero de telefono para armarlo en una sola


In [18]:
df_phones_sorted = df_phones_sorted.select(concat(df_phones_sorted.prefix_phone_id,df_phones_sorted.phone_area_id,df_phones_sorted.phone_exchange_id,df_phones_sorted.phone_line_id).alias('Full_Phone'),'customer_id','last_change_date')

In [19]:
df_phones_sorted.show()

+-------------+-----------+----------------+
|   Full_Phone|customer_id|last_change_date|
+-------------+-----------+----------------+
| 542664697946|   00000007|      2022-09-01|
| 543412847321|   00000039|      2022-09-20|
| 541125064159|   00000044|      2022-10-03|
| 543815909885|   00000381|      2022-07-19|
| 542975296284|   00000442|      2022-07-20|
| 542974729337|   00000442|      2022-07-20|
| 541166793207|   00001419|      2022-11-01|
| 541138700150|   00001939|      2022-09-21|
| 542994477116|   00002707|      2022-08-17|
| 543424662478|   00002790|      2022-11-18|
| 543424883620|   00002790|      2022-10-08|
|5435415988799|   00004287|      2022-08-29|
| 541124084447|   00004724|      2022-11-04|
| 542613862762|   00005527|      2022-10-03|
| 542615904192|   00005527|      2022-10-03|
| 543854065887|   00007932|      2022-09-29|
| 543489493578|   00011850|      2022-10-27|
| 541125781080|   00011850|      2022-10-24|
| 543401534381|   00012051|      2022-10-06|
| 54116114

Utilizamos windows para realizar las particiones de costumer_id

In [20]:
window = Window.partitionBy(df_phones_sorted.customer_id).orderBy(desc(df_phones_sorted.last_change_date))

In [21]:
df_phone = df_phones_sorted.withColumn('row_num', row_number().over(window))

In [22]:
df_phone = df_phone.filter(df_phone.row_num <= 3)

In [23]:
df_phone.show()

+-------------+-----------+----------------+-------+
|   Full_Phone|customer_id|last_change_date|row_num|
+-------------+-----------+----------------+-------+
| 542664697946|   00000007|      2022-09-01|      1|
| 543815909885|   00000381|      2022-07-19|      1|
| 543424662478|   00002790|      2022-11-18|      1|
| 543424883620|   00002790|      2022-10-08|      2|
|5435415988799|   00004287|      2022-08-29|      1|
| 543854065887|   00007932|      2022-09-29|      1|
| 543489493578|   00011850|      2022-10-27|      1|
| 541125781080|   00011850|      2022-10-24|      2|
| 543401534381|   00012051|      2022-10-06|      1|
| 542216208511|   00013498|      2022-08-22|      1|
| 541165050605|   00014664|      2022-10-06|      1|
| 541161577947|   00041884|      2022-07-28|      1|
| 541165187983|   00048225|      2022-07-08|      1|
| 541169951912|   00052103|      2022-09-27|      1|
| 543364577255|   00056407|      2022-10-17|      1|
| 543412740967|   00058519|      2022-10-26|  

Realizamos la tabla pivot

In [24]:
df_pivot_phone = df_phone.groupBy('customer_id').agg(collect_list('Full_Phone').alias('last_3_changes_list'))

In [25]:
df_pivot_phone = df_pivot_phone.selectExpr('customer_id', 'last_3_changes_list[0] as Phone_1', 'last_3_changes_list[1] as Phone_2', 'last_3_changes_list[2] as Phone_3')

In [26]:
df_pivot_phone.show()

+-----------+-------------+------------+-------+
|customer_id|      Phone_1|     Phone_2|Phone_3|
+-----------+-------------+------------+-------+
|   00000007| 542664697946|        null|   null|
|   00000381| 543815909885|        null|   null|
|   00002790| 543424662478|543424883620|   null|
|   00004287|5435415988799|        null|   null|
|   00007932| 543854065887|        null|   null|
|   00011850| 543489493578|541125781080|   null|
|   00012051| 543401534381|        null|   null|
|   00013498| 542216208511|        null|   null|
|   00014664| 541165050605|        null|   null|
|   00041884| 541161577947|        null|   null|
|   00048225| 541165187983|        null|   null|
|   00052103| 541169951912|        null|   null|
|   00056407| 543364577255|        null|   null|
|   00058519| 543412740967|        null|   null|
|   00058909| 541125222584|        null|   null|
|   00064043| 542976219525|542975133355|   null|
|   00064339| 542966425661|        null|   null|
|   00071569| 543584

Reemplazamos null por ---

In [27]:
df_pivot_phone = df_pivot_phone.na.fill('---')
df_pivot_phone.show()

+-----------+-------------+------------+-------+
|customer_id|      Phone_1|     Phone_2|Phone_3|
+-----------+-------------+------------+-------+
|   00000007| 542664697946|         ---|    ---|
|   00000381| 543815909885|         ---|    ---|
|   00004287|5435415988799|         ---|    ---|
|   00007932| 543854065887|         ---|    ---|
|   00011850| 543489493578|541125781080|    ---|
|   00012051| 543401534381|         ---|    ---|
|   00013498| 542216208511|         ---|    ---|
|   00014664| 541165050605|         ---|    ---|
|   00041884| 541161577947|         ---|    ---|
|   00048225| 541165187983|         ---|    ---|
|   00052103| 541169951912|         ---|    ---|
|   00056407| 543364577255|         ---|    ---|
|   00058519| 543412740967|         ---|    ---|
|   00058909| 541125222584|         ---|    ---|
|   00064043| 542976219525|542975133355|    ---|
|   00064339| 542966425661|         ---|    ---|
|   00071569| 543584112643|         ---|    ---|
|   00077558| 541151

In [28]:
df_customer = spark.read.load('sample_data/customer_basics_bootcamp.snappy.parquet', sep=',', inferschema='true', header='true')
df_phones = spark.read.load('sample_data/phones_bootcamp.snappy.parquet', sep=',', inferschema='true', header='true')

In [29]:
df_phones.show()

+-----------+--------------+-------------------+-----------------+----------+----------------+---------------+-------------+-------------------+-----------------+-------------+--------------+---------------+-----------------------+------------------+---------------------+--------------------+----------+-----------+--------------------------+---------------------+--------------------+---------------------+----------------+-------------------------+-------------------------+------------------+-------------------+-----------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+---------------------+
|customer_id|phone_use_type|address_sequence_id|phone_sequence_id|phone_type|phone_country_id|prefix_phone_id|phone_area_id|cellph

#Agregar una nueva columna a los DataFrame de contactos, indicando el contact_type según corresponda (address, email, phone)

In [40]:
from pyspark.sql.functions import lit

In [41]:
df_phones = df_phones.withColumn('contact_type', lit('phone'))

In [42]:
df_phones.show()

+-----------+--------------+-------------------+-----------------+----------+----------------+---------------+-------------+-------------------+-----------------+-------------+--------------+---------------+-----------------------+------------------+---------------------+--------------------+----------+-----------+--------------------------+---------------------+--------------------+---------------------+----------------+-------------------------+-------------------------+------------------+-------------------+-----------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+---------------------+------------+
|customer_id|phone_use_type|address_sequence_id|phone_sequence_id|phone_type|phone_country_id|prefix_phone_id|phone_a

In [35]:
df_emails = spark.read.load('sample_data/emails_bootcamp.snappy.parquet', sep=',', inferschema='true', header='true')

In [44]:
df_emails = df_emails.withColumn('contact_type', lit('e-mail'))
df_emails.show()

+-----------+---------+-------------------+--------------+----------+------------------+--------------------+-----------------+--------------+-------------------+----------------+--------------------------+--------------+----------------+-------------------+--------------------+-----------------------+---------------------+--------------------------+-------------------+----------------+------------+
|customer_id|role_type|address_sequence_id|residence_type|email_type|primary_email_type|          email_desc|email_domain_type|encripted_type|field_length_number|   comments_desc|customer_email_status_type|email_app_type|register_user_id|last_change_user_id|last_change_hms_date|last_change_terminal_id|operational_load_date|customer_email_status_date|registry_entry_date|last_change_date|contact_type|
+-----------+---------+-------------------+--------------+----------+------------------+--------------------+-----------------+--------------+-------------------+----------------+---------------

In [37]:
df_address = spark.read.load('sample_data/address_bootcamp.snappy.parquet', sep=',', inferschema='true', header='true')

In [45]:
df_address = df_address.withColumn('contact_type', lit('address'))
df_address.show()

+-----------+-----------------------+-------------------+--------------+--------------------+------------------+---------------------------+-----------------+-------------+---------------------+--------------------+-----------------------+----------+---------------+-----------+------------------+----------------------+-------------------------+------------------+---------------------+------------------------------+-----------------------+--------------------+---------------------+------------------+-------------------------+-------------------------+------------------+-----------------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+-----------------+---------------------+------------+
|cu